# LLM Data Cleansing Pipeline

#### Loading Environment Variables and Initialize API Key

In [1]:
from dotenv import load_dotenv
import os

# loading .env file
load_dotenv()

# setting OPENAI_API_KEY
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

#### Initialize LLM

In [2]:
import openai
from langchain_openai import ChatOpenAI


# Setting model
llm_model = "gpt-3.5-turbo" 

# Initialize OpenAI LLM with LangChain
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name=llm_model,
    #temperature=0.7,   # Control randomness: 0.0 is deterministic, 1.0 is very creative
    #max_tokens=1500    # Max tokens in the response
)

#### Initialize Prompt

In [3]:
# Load prompt from prompt.txt file
with open('prompt.txt', 'r') as file:
    prompt = file.read()

print('\nPreview Prompt:\n')
print(prompt)


Preview Prompt:

You are tasked with converting the following durations to days. These durations are a free-form text 
field on an 811 OneCall Ticket.  The Duration is how long the job should last. There may be misspellings 
or errors. If you see an estimate or range, use the highest. The minimum number of days should be 1, 
if something is 1 hour, it should be 1 day. Anything that cannot convert to a day, is error. For full days, 
enter the exact number of days. For any part of a day mentioned (like hours or half days), count each as a 
full day if the activity spans multiple days, reflecting the occupation of each calendar day. 


First column, “duration”, will be the original value (leave it exactly as it was entered). 

Second column, “days”, will be standardized amount in days (integer only).   

Third column, "is_estimate",  will be binary (1 for yes, 0 for no). This used for anything that could 
be an estimate based on ranges or if language that could suggest an estimate is pre

#### Initialize Data

Loading <code>llm_unique_data.csv</code>, which has duplicate values removed to save on unnecessary computational costs.

Using tiktoken to determin total tokens to be sent.

In [4]:
import pandas as pd
import tiktoken

# Load the CSV data and assign to DataFrame
csv_file_path = 'data/llm_unique_data.csv'
df = pd.read_csv(csv_file_path)

# Setting the DURATION column to llm_data
llm_data = df['DURATION']

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

# Tokenize each entry in the DURATION column and count tokens
llm_data_tokens = llm_data.apply(lambda x: len(tokenizer.encode(str(x))))

# Calculate the total number of tokens
total_tokens = llm_data_tokens.sum()

# Checking output 
print(f'\nTotal Rows: {len(llm_data)}')
print(f'Total Tokens: {total_tokens}')
print('\nData Preview:')
display(llm_data.head())



Total Rows: 1356
Total Tokens: 5318

Data Preview:


Total Rows: 1356
Total Tokens: 5318

Data Preview:


0    NOT APPLICABLE
1           14 DAYS
2             1 DAY
3            5 DAYS
4           2 WEEKS
Name: DURATION, dtype: object

#### Define Function to Batch <code>llm_data</code> Based on Tokens

In [5]:
import tiktoken

max_tokens = 3000

def split_data_by_token_limit(data, prompt, max_tokens=max_tokens):

    """Splits data into batches that stay within the token limit."""
    
    current_batch = []
    prompt_tokens = len(tokenizer.encode(prompt))  # Get the token count for the prompt
    current_tokens = prompt_tokens  # Start the token count for the batch with the prompt tokens
    
    for row in data:
        row = str(row)  # Ensure that the row is treated as a string
        row_tokens = len(tokenizer.encode(row))
        
        # Check if adding the current row would exceed the max_tokens limit
        if current_tokens + row_tokens > max_tokens:
            yield current_batch  # Return the current batch when max_tokens is reached
            current_batch = []  # Start a new batch
            current_tokens = prompt_tokens  # Reset token count with the prompt tokens included for the new batch
        
        current_batch.append(row)
        current_tokens += row_tokens  # Add the row's token count to the current batch
    
    if current_batch:  # Yield the last batch if there's any data left
        yield current_batch

#### Process Each Batch

*** eventually want to make role more specific ***

In [6]:
# Initialize the list to store results
results = []

# Loop through each batch
for batch in split_data_by_token_limit(llm_data, prompt):
    # Prepare the data from the current batch
    batch_text = ', '.join(batch)
    
    # Combine the prompt and batch data into a single message format
    messages = [
        {"role": "user", "content": f"{prompt}\nData: {batch_text}"}
    ]

    # Call the LLM using the .invoke() method
    try:
        response = llm.invoke(messages)
        
        # Extract the content (the results returned by the LLM)
        content = response.content
        
        # Append the content to the results list
        results.append(content)
    
    except Exception as e:
        print(f"Error processing batch: {e}")

#### Convert Responses to DataFrame

*** add preprocessing steps to avoid having to skip bad lines ***

In [7]:
import pandas as pd
import io

# Initialize an empty list to store DataFrames
df_list = []

# Process each CSV string in results
for content in results:
    try:
        # Read the CSV-formatted string into a DataFrame
        df = pd.read_csv(io.StringIO(content), on_bad_lines='skip') # Preprocess to avoid skipping
        df_list.append(df)
    
    except Exception as e:
        print(f"Error parsing result: {e}")


# Concatenate all DataFrames if any
if df_list:
    df_results = pd.concat(df_list, ignore_index=True)
    display(df_results)

else:
    print("No valid results to convert into a DataFrame.")

,duration,days,is_estimate
0,NOT APPLICABLE,1.0,0.0
1,14 DAYS,14.0,0.0
2,1 DAY,1.0,0.0
3,5 DAYS,5.0,0.0
4,2 WEEKS,14.0,1.0
...,...,...,...
932,68.1 HRS,69.0,0.0
933,40 HOURS,40.0,0.0
934,7 OR 8 DAYS,8.0,0.0
935,5-20 DAYS,20.0,1.0


#### Examples of Bad Lines

In [11]:
# Combine all results into a single string
all_results = "\n".join(results)

# Split the combined results into lines
lines = all_results.split('\n')

# Inspect the problematic lines
print(f"Line 159: {lines[158]}")  # Indexing starts from 0
print(f"Line 415: {lines[414]}")

Line 159: 1 WEEK,,7,0
Line 415: 30 MINS - 1 HOU,1,1
Line 159: 1 WEEK,,7,0
Line 415: 30 MINS - 1 HOU,1,1
